# Churn Analysis - Model Building

In [14]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [15]:
df=pd.read_csv("tel_churn.csv")
df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,0,56.95,1889.50,0,0,1,1,0,1,...,0,0,0,1,0,0,1,0,0,0
2,2,0,53.85,108.15,1,0,1,1,0,1,...,0,0,0,1,1,0,0,0,0,0
3,3,0,42.30,1840.75,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,4,0,70.70,151.65,1,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0


In [16]:
df=df.drop('Unnamed: 0',axis=1)

1. Logistic Regression

In [17]:
y = df['Churn'].values
X = df.drop(columns = ['Churn'])

In [18]:
from sklearn.preprocessing import MinMaxScaler
features = X.columns.values
scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X))
X.columns = features

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [20]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(max_iter=1000)
result = LR.fit(X_train, y_train)

In [21]:
from sklearn import metrics
prediction_test = LR.predict(X_test)

In [22]:
print (metrics.accuracy_score(y_test, prediction_test))

0.8061611374407583


2. Random Forest

In [35]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
model_rf = RandomForestClassifier(n_estimators=1000 , oob_score = True, n_jobs = -1,
                                  random_state =50, max_features = "auto",
                                  max_leaf_nodes = 30)
model_rf.fit(X_train, y_train)

prediction_test = model_rf.predict(X_test)
print (metrics.accuracy_score(y_test, prediction_test))

C:\ProgramData\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


0.8102345415778252


3. Support Vector Machine

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

In [31]:
from sklearn.svm import SVC

svc_model = SVC(random_state = 1)
svc_model.fit(X_train,y_train)
predict_y = svc_model.predict(X_test)
accuracy_svc = svc_model.score(X_test,y_test)
print("SVM accuracy is :",accuracy_svc)

SVM accuracy is : 0.8173418621179815


In [33]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,predict_y))  

[[974  68]
 [189 176]]


4. XG Boost

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

In [39]:
from xgboost import XGBClassifier
modelXGB = XGBClassifier()
modelXGB.fit(X_train, y_train)
preds = modelXGB.predict(X_test)
metrics.accuracy_score(y_test, preds)

0.8052594171997157

5. ADA Boast

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

In [37]:
from sklearn.ensemble import AdaBoostClassifier
modelADA = AdaBoostClassifier()
modelADA.fit(X_train,y_train)
preds = modelADA.predict(X_test)
metrics.accuracy_score(y_test, preds)

0.8137882018479033

6.GradientBoostingClassifier

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

In [47]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)
print("Gradient Boosting Classifier", metrics.accuracy_score(y_test, gb_pred))

Gradient Boosting Classifier 0.8237384506041222


In [49]:
from imblearn.combine import SMOTEENN

In [52]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(X, y)

In [57]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [58]:
gb_smote = GradientBoostingClassifier()

In [60]:
gb_smote.fit(xr_train,yr_train)
yr_predict = gb_smote.predict(xr_test)
model_score_r = gb_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9512779552715654
              precision    recall  f1-score   support

           0       0.96      0.93      0.94       529
           1       0.95      0.97      0.96       723

    accuracy                           0.95      1252
   macro avg       0.95      0.95      0.95      1252
weighted avg       0.95      0.95      0.95      1252



Clearly Gradient BOASTING CLASSIFIER IS A WINNER!

Pickling the model

In [62]:
import pickle

filename = 'model.sav'
pickle.dump(gb_smote,open(filename, 'wb'))
load_model = pickle.load(open(filename, 'rb'))

In [65]:

model_score = load_model.score(xr_test, yr_test)

In [66]:

model_score

0.9512779552715654